In [3]:
%pip install --upgrade --quiet devtools

Note: you may need to restart the kernel to use updated packages.


In [7]:
from dotenv import load_dotenv
import json
from pinecone import Pinecone

# from sentence_transformers import SentenceTransformer
import re
from langchain_openai import OpenAIEmbeddings

load_dotenv()

# model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="mps")
embeddings = OpenAIEmbeddings()

In [12]:
filename = "./sa-catalog-v3.json"

pc = Pinecone()
index = pc.Index("catalog-v2")

i = j = 0
batch_size = 100
upserted_count = 0
texts, ids, embeds, metadata = [], [], [], []

with open(filename) as file:
    for line in file:
        if j % 1000 == 0:
            print(f"Iteration: {j}. Upserted: ~{upserted_count}")
        if i % batch_size == 0 and texts:
            embeds = embeddings.embed_documents(texts)
            upserted_count += len(embeds)
            index.upsert(vectors=zip(ids, embeds, metadata))
            texts, ids, embeds, metadata = [], [], [], []
        doc = json.loads(line.rstrip())
        if (
            doc.get("title_en")
            and doc.get("seller_price")
            and doc.get("sku_config")
        ):
            specs = doc.get("specifications") or []
            if not specs:
                continue
            specs_text = ""
            specs = json.loads(specs)
            model_year = None
            for spec in specs:
                k = spec.get("name")
                v = spec.get("value")
                specs_text += f"{k} : {v}" + "\n"
                if k == "Model Year":
                    model_year = v

            ids.append(doc["sku_config"])

            sku = doc["sku_config"]
            title = doc["title_en"]
            offer_price = doc["seller_price"]
            description = doc.get("description", "")
            highlights = doc.get("highlights", "")
            image_url = f"{doc.get('image_url')}?width=240" if doc.get("image_url") else "Unavailable"
            product_url = f"https://www.{doc.get('product_url')}" if doc.get('product_url') else "Unavailable"
            rating = doc.get("product_rating", "Unavailable")
            warranty = doc.get("warranty", "Unavailable")
            on_deal = doc.get("deal_flag") or 0
            instock = (
                1
                if doc.get("instock_flag", "Unavailable").strip().lower() == "true"
                else 0
            )
            product_url = f"https://www.noon.com/saudi-en/xyz/{sku}/p" 

            m = {
                "image_url": image_url,
                "product_url": product_url,
                "title": title,
                "price": offer_price,
                "rating": rating,
                "in_stock": instock,
                "sku": sku,
            }
            text = f"""
                Sku: {sku}
                Title: {title}. 
                Price: {offer_price}. 
                Available: {instock}. 
                Description: {description}. 
                Highlights: {highlights}. 
                Rating: {rating}
                image_url: {image_url}
                product_url: {product_url} 
                Warranty: {warranty}. 
                Specifications: {specs_text}. 
                is_on_deal: {on_deal}
                """.strip()
            if model_year:
                text += f"Model Year: {model_year}"
            text = re.sub(r"\s+", " ", text)
            m["text"] = text
            metadata.append(m)
            texts.append(text)
            i += 1
        j += 1

Iteration: 0. Upserted: ~0
Iteration: 1000. Upserted: ~900
Iteration: 2000. Upserted: ~1900


KeyboardInterrupt: 